# День 02 – Машинное обучение

В начале импортируй библиотеки, которые понадобяться нам в работе.

In [1]:
import pandas as pd
import numpy as np

import shap
import sklearn
import catboost

import seaborn as sns
import matplotlib.pyplot as plt;

Если ячейка выше у нас **не запускается**, то скорее всего у нас **не установлена какая-либо библиотека**.  **Чтобы установить библиотеку напиши**:
`pip install scikit-learn` или `pip install catboost` или `pip install shap` 

In [2]:
sns.set_style("darkgrid") #у графиков будет красивая серая подложка

Давай загрузим наши данные.

## Загрузка данных

In [3]:
train = pd.read_csv("../../data/v2/day02/dataset_01_06.csv") # Загружаем данные из csv
train = train.set_index("client_id") # Устанавливаем индекс клиента
train = train.drop_duplicates() # Убираем дубликаты

In [4]:
train.head(5) # Смотрим первые 5 строк в таблице

,prepay_ind_p1m,latitude_1m,building_type_key_1m,deferred_pay_1m,pay_count_p3m,crm_init_count_d1m,crm_out_count_p3m,prepay_ind_d2m,basic_day_debt_bad_max_d1m,bad_debt_p1m,...,avg_view_genre_dosug_2m,is_multiscreen_p1m,avg_view_other_p1m,count_purchase_1m,avg_view_multi_1m,total_duration_1m,avg_view_adult_1m,avg_view_category_18_d3m,avg_view_tvmarket_p2m,avg_view_category_0_2m
client_id,,,,,,,,,,,,,,,,,,,,,
161291,0.0,51.363170,377945938.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,18.37,1470.87,0.0,0.0,0.00,0.0
87256,0.0,50.547399,377945938.0,NaN,3.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
128419,0.0,51.738616,377945938.0,1.0,8.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,77.63,10.0,88.69,15110.70,0.0,0.0,0.00,0.0
198395,0.0,50.087963,377945938.0,NaN,3.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
194340,0.0,51.700059,377945938.0,NaN,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,173.79,7342.57,0.0,0.0,4.67,0.0


С помощью метода `.shape`. Можно вывести размерность датасета.

In [5]:
train.shape

(32092, 3647)

## Утечки данных

В наших данных о клиентах есть признаки, которое могут быть *утечками данных*. Утечка данных может привести к 
созданию чрезмерно оптимистичных, а то и вовсе бесполезных прогнозных моделей. Утечка данных - это когда 
для создания модели используется информация, которая не будет доступна на этапе эксплуатации модели. Например, такие признаки с приставками `hgid`, `hflat`.

Для того, чтобы получить признаки c приставками `hgid` воспользуемся кодом ниже:

In [6]:
# Из всех колонок возьмем только колонки, которые начинаются с "hgid"
lick_columns = []
lick_columns.extend([col for col in train.columns if col.startswith('hgid')])

In [7]:
lick_columns

['hgid_hm_1m', 'hgid_1m']

Теперь в переменной `lick_columns` храняться признаки, которые могут быть утечками в данных. Добавим в этот список признаки с приставкой `hflat`.

In [8]:
lick_columns.extend([col for col in train.columns if col.startswith('hflat')])

In [9]:
lick_columns

['hgid_hm_1m', 'hgid_1m', 'hflat_1m']

## Задание 1

Удалите признаки с приставками `charg_inst`, `charg_sale`, `hgid`, `hflat`, `hlid`, а также данные о координатах `latitude_1m` и `longitude_1m`. В удалении признаков вам поможет метод [.drop()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)

Выведи размерность получившейся таблицы

In [10]:
lick = [x for x in train.columns if x.startswith('charg_inst') or x.startswith('charg_sale')]

In [11]:
lick.extend([_ for _ in train.columns if _.startswith('hgid') or _.startswith('hflat') or _.startswith('hlid')])
lick.extend(['latitude_1m', 'longitude_1m'])

In [12]:
train = train.drop(columns=lick)

In [13]:
train.shape

(32092, 3587)

## Разделение на признаки и предсказываемую величину

В машинном обучении, когда мы говорим о разделении на признаки и предсказываемую величину, мы имеем в виду следующее:

1. Признаки (features): Это переменные или характеристики, которые используются для прогнозирования. Например, если у нас есть данные о жилье, признаками могут быть количество спален, площадь, район и т.д.

2. Предсказываемая величина (target): Это величина, которую мы пытаемся предсказать с помощью модели машинного обучения. В задачах регрессии это может быть непрерывная величина, такая как цена жилья, вес человека и т.д. В задачах классификации это может быть категория или метка, например, "клиент уйдет" или "клиент останется".

При обучении модели машинного обучения мы используем признаки для предсказания целевой переменной. Таким образом, разделение на признаки и предсказываемую величину является важным этапом в подготовке данных для построения модели.

## Задание 2

Теперь разберемся, что в каждой из задач для нас будет являться признаком (Х), а что предсказываемой переменной (Y).

Раздели датасет `train` на признаки и предсказываемую величину. Признаки сохрани в переменную `X`, предсказываемую величину в переменную `Y`. Выведи размерность `X` и `Y`.

In [14]:
Y = train['label']
X = train.drop(columns=['label'])

In [15]:
Y.shape

(32092,)

In [16]:
X.shape

(32092, 3586)

## Разделение признаков на категориальные и численные

Разделение признаков на категориальные и численные важно в контексте подготовки данных для модели машинного обучения по нескольким причинам:

1. Обработка данных: Категориальные признаки представляют собой качественные переменные, которые могут принимать ограниченное количество значений (например, цвет, тип жилья и т.д.), в то время как численные признаки представляют собой количественные переменные (например, возраст, цена, площадь и т.д.). Обработка этих двух типов признаков может потребовать различных методов: категориальные признаки могут потребовать кодирования (например, One-Hot Encoding), в то время как численные признаки могут потребовать масштабирования или нормализации.

2. Выбор модели: Некоторые модели машинного обучения могут требовать явного указания типа признаков. Например, некоторые модели, такие как деревья решений, могут автоматически обрабатывать категориальные признаки, в то время как другие модели, такие как линейная регрессия, могут требовать предварительного преобразования категориальных признаков в численные.

3. Интерпретация результатов: Различные типы признаков могут оказывать разное влияние на модель. Например, при интерпретации коэффициентов в линейной регрессии, численные признаки могут иметь более прямое воздействие на предсказываемую величину, чем категориальные.

Таким образом, разделение признаков на категориальные и численные позволяет более эффективно обрабатывать данные и выбирать подходящие методы анализа и моделирования для конкретной задачи машинного обучения.

## Задание 3

Теперь нам потребуется разделить наши признаки на численные и категориальные. В этом нам поможет функция метод [.select_dtypes()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.select_dtypes.html). Численные переменные имеют тип `'number'`, а категориальные `'object'`.
Численные переменные сохрани в переменную X_num, категориальные в переменную X_cat.

Выведи размерность этих таблиц.

In [17]:
X_cat = X.select_dtypes('object')

In [18]:
X_num = X.select_dtypes('number')

In [19]:
X_cat.shape

(32092, 8)

In [20]:
X_num.shape

(32092, 3578)

## Задание 4

Некоторые алгоритмы машинного обучения чувствительны к пропускам данных, поэтому нам придется заполнить отсутствующие 
данные чем то заполнить. Воспользуйся методом [.fillna()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html)
для того, чтобы заполнить пропуски в численных признаках значением 0.

Затем выполни этот код `X_num.isna().sum().sum()`

In [21]:
X_num = X_num.fillna(value=0)

In [22]:
X_num.isna().sum().sum()

0

## One-Hot Encoding

One-Hot Encoding — это процесс преобразования категориальных (номинальных) признаков в числовой формат. Зачем нам важно применять One-Hot Encoding:

1. Совместимость с моделями машинного обучения: Многие алгоритмы машинного обучения требуют, чтобы все данные были числовыми. One-Hot Encoding позволяет преобразовать категориальные данные в форму, которую модели могут использовать для обучения.

2. Избежание неявных порядковых отношений: При использовании One-Hot Encoding мы избегаем создания неявных порядковых отношений между категориями. Кодирование категорий числами (например, 1, 2, 3) может ввести модель в заблуждение, будто некоторые категории имеют порядок или значимость.

3. Предотвращение "расстояний" между категориями: При использовании One-Hot Encoding расстояние между любыми двумя закодированными категориями равно 1. Это помогает избежать неправильных выводов о сходстве или различии между категориями.


<center><img src="../../DSpy02.ID_1421592/misc/images/one-hote-encoding.png"/>

## Задание 5

Примени преобразования **One-Hot Encoding** для категориальных признаков. В этом тебе поможешь функция [pd.get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html)

Выведи размерность получившейся таблицы

In [23]:
X_cat = pd.get_dummies(X_cat)

In [24]:
X_cat.shape

(32092, 24895)

## Задание 6

Теперь уже преобразованные части требуется снова объединить в единый датасет. Соедини переменные `X_num` и `X_cat` в переменную `X`. Воспользуйся методом [.merge()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html).
Объединение проведи по индексам таблицы.

Выведи размерность получившейся таблицы

In [25]:
X = X_cat.merge(X_num, left_index=True, right_index=True)

In [26]:
X_num.shape

(32092, 3578)

In [27]:
X_cat.shape

(32092, 24895)

In [28]:
X.shape

(32092, 28473)

## Разделение на Train, Test части

Разделение на тренировочную и тестовую части является важным шагом при обучении моделей машинного обучения. Этот процесс помогает оценить производительность модели на данных, которые она ранее не видела. Обычно данные разделяются в соотношении 70-80% на тренировочную выборку и 20-30% на тестовую выборку.

Разделение на тренировочную и тестовую части помогает избежать переобучения (overfitting) и оценить способность модели к обобщению на новые данные.

## Задание 7

Раздели итоговый датасет на train и test части. В этом поможет функция [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). Разделение проведи с параметрами `test_size=0.2`, `random_state=21`.

Выведи размерности переменных `X_train` и `X_test`

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=21)

In [31]:
X_train.shape

(25673, 28473)

In [32]:
X_test.shape

(6419, 28473)

## Обучение модели

Теперь у нас все готово для обучения!
Мы попробуем обучить модель - градиентный бустинг. Градиентный бустинг — это метод машинного обучения, который используется для построения прогностических моделей, таких как регрессия или классификация. Он основан на идее построения ансамбля слабых моделей, обычно деревьев решений, и объединении их в сильную модель.

Теперь попробуй использовать алгоритм градиентного бустинга для обучения нашей модели предсказания оттока. Для этого воспользуйся алгоритмом `CatBoostRegressor` из библиотеки `catboost`.

In [33]:
from catboost import CatBoostClassifier, MetricVisualizer
from sklearn.metrics import roc_auc_score, accuracy_score

In [34]:
model = CatBoostClassifier(iterations=2000, eval_metric='AUC', train_dir="base").fit(
    X_train, 
    y_train, 
    verbose=100,
    eval_set=(X_test, y_test),
)

Learning rate set to 0.052252
0:	test: 0.5027181	best: 0.5027181 (0)	total: 141ms	remaining: 4m 41s
100:	test: 0.6603477	best: 0.6603477 (99)	total: 6.54s	remaining: 2m 2s
200:	test: 0.6646554	best: 0.6650932 (191)	total: 12.8s	remaining: 1m 54s
300:	test: 0.6657723	best: 0.6664208 (285)	total: 19.2s	remaining: 1m 48s
400:	test: 0.6642734	best: 0.6664208 (285)	total: 25.3s	remaining: 1m 40s
500:	test: 0.6639199	best: 0.6664208 (285)	total: 31.2s	remaining: 1m 33s
600:	test: 0.6646915	best: 0.6664208 (285)	total: 37.1s	remaining: 1m 26s
700:	test: 0.6646452	best: 0.6664208 (285)	total: 42.9s	remaining: 1m 19s
800:	test: 0.6660442	best: 0.6664208 (285)	total: 48.6s	remaining: 1m 12s
900:	test: 0.6663365	best: 0.6664208 (285)	total: 54.3s	remaining: 1m 6s
1000:	test: 0.6669388	best: 0.6669619 (969)	total: 59.9s	remaining: 59.8s
1100:	test: 0.6689360	best: 0.6689441 (1078)	total: 1m 5s	remaining: 53.6s
1200:	test: 0.6683453	best: 0.6693357 (1180)	total: 1m 11s	remaining: 47.4s
1300:	test: 

С помощью функции `MetricVisualizer` мы можем визуализировать как обучалась модель.

In [35]:
MetricVisualizer(["base"]).start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

## Задание 8

Рассчитай точность обученной модели. Для этого с помощью метода `.predict(X_test)` сделай предсказание на тестовой 
выборке `X_test`. Затем передай предсказанные классы и истинные значения оттока в функцию `accuracy_score`. 

Не подозрительный ли получился результат? Рассчитай долю клиентов, которые **не уйдут**.

In [36]:
pred = model.predict(X_test)

In [37]:
accuracy_score(y_test, pred)

0.9630783611154385

In [38]:
1 - (y_test.sum() / len(y_test))

0.9629225736095965

## Дизбаланс классов

При работе с несбалансированными классами использование метрики точности может привести к искаженным результатам оценки производительности модели. При дисбалансе классов модель может быть склонна предсказывать чаще наиболее представленный класс, что приведет к высокой точности для этого класса, но низкой для менее представленного класса. Точность может быть высокой, даже если модель предсказывает только доминирующий класс, игнорируя менее представленный класс. Это может создать ложное впечатление о качестве модели.

Для более информативной оценки модели в случае дисбаланса классов рекомендуется использовать другие метрики, такие как ROC AUC, F1-мера, точность и полнота. Эти метрики учитывают как правильность классификации положительных и отрицательных классов, так и способность модели находить все положительные случаи в данных.

<center><img src="../../DSpy02.ID_1421592/misc/images/metrics.png"/>

ROC AUC (Receiver Operating Characteristic Area Under the Curve) — это метрика, которая широко используется для оценки качества бинарной классификации моделей машинного обучения. Вот несколько причин, почему использовать ROC AUC полезно:

1. Интерпретируемость: ROC AUC предоставляет понятную интерпретацию качества классификационной модели. Значение ROC AUC находится между 0 и 1, где 1 означает идеальную модель, а 0.5 — модель, которая предсказывает случайно.

2. Устойчивость к дисбалансу классов: ROC AUC не зависит от дисбаланса классов в данных, что делает его хорошей метрикой для несбалансированных наборов данных.

<center><img src="../../DSpy02.ID_1421592/misc/images/ROC-AUC.png"/>

## Задание 9

Рассчитай метрику ROC-AUC. Для этого с помощью метода `.predict_proba(X_test)[:,1]` сделай предсказание на тестовой 
выборке `X_test`. Затем передай вероятности класса и истинные значения оттока в функцию `roc_auc_score`. 

In [39]:
proba = model.predict_proba(X_test)[:,1]

In [40]:
roc_auc_score(y_test, proba)

0.6795628783789847